In [1]:
from collections import Counter
from konlpy.tag import *
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [2]:
corpus= [
    "나는 너를 사랑한다.",
    "나는 나를 사랑한다.",
    "나는 나를 증오한다.",
    "나는 너를 증오한다.",
    "너는 너를 증오한다."
    ]
df=pd.DataFrame({"id": range(len(corpus)),"corpus": corpus,"label":[1,0,0,1,1]
})                 
df



s=" ".join(df.corpus)

print('All string :' ,s)

# posToUse=["NNP","NNG","MAG","NP","VV","VV+EF","IC"]
posToUse=["NNP","NNG","NP","NNB"]

def getTokens(s):
    global posToUse
    return [ i[0] for i in  Mecab().pos(s) if i[1] in posToUse ] 
    
print('All tockents:', getTokens(s))

posToUse=["NNP","NNG","NP","NNB"]
stopwords=["놈","18"]
def getToken(s,pos=posToUse,stopword=stopwords):
    return [ w for w,t in  Mecab().pos(s) if t in pos and w not in stopword ] 
vect = CountVectorizer(tokenizer=getToken)
vect.fit(getTokens(s))
print('Dictionary : ',vect.vocabulary_)
cd=pd.DataFrame(vect.transform([" ".join(getTokens(i)) for i in df.corpus]).toarray())
data=pd.concat([df,cd], axis=1)
data

All string : 나는 너를 사랑한다. 나는 나를 사랑한다. 나는 나를 증오한다. 나는 너를 증오한다. 너는 너를 증오한다.
All tockents: ['나', '너', '사랑', '나', '나', '사랑', '나', '나', '증오', '나', '너', '증오', '너', '너', '증오']
Dictionary :  {'나': 0, '너': 1, '사랑': 2, '증오': 3}


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


,id,corpus,label,0,1,2,3
0,0,나는 너를 사랑한다.,1,1,1,1,0
1,1,나는 나를 사랑한다.,0,2,0,1,0
2,2,나는 나를 증오한다.,0,2,0,0,1
3,3,나는 너를 증오한다.,1,1,1,0,1
4,4,너는 너를 증오한다.,1,0,2,0,1


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop


x_train=data.iloc[:,3:11].to_numpy()
#y_train=data.label.to_numpy().reshape([-1,1])
y_train=keras.utils.to_categorical(data.label,2)

In [4]:
print(x_train.shape)
print(y_train.shape)

(5, 4)
(5, 2)


In [5]:
model = Sequential()
model.add(Dense(14, activation='tanh', input_shape=(x_train.shape[1],)))
model.add(Dense(14, activation='tanh'))
model.add(Dense(4, activation='tanh'))
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 14)                70        
_________________________________________________________________
dense_1 (Dense)              (None, 14)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 60        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 10        
Total params: 350
Trainable params: 350
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_train, y_train))


Epoch 1/10
5/5 [==============================] - 0s 36ms/step - loss: 0.7742 - accuracy: 0.4000 - val_loss: 0.7292 - val_accuracy: 0.4000
Epoch 2/10
5/5 [==============================] - 0s 6ms/step - loss: 0.7296 - accuracy: 0.4000 - val_loss: 0.7066 - val_accuracy: 0.6000
Epoch 3/10
5/5 [==============================] - 0s 6ms/step - loss: 0.6990 - accuracy: 0.6000 - val_loss: 0.6835 - val_accuracy: 0.6000
Epoch 4/10
5/5 [==============================] - 0s 7ms/step - loss: 0.6776 - accuracy: 0.6000 - val_loss: 0.6612 - val_accuracy: 0.6000
Epoch 5/10
5/5 [==============================] - 0s 7ms/step - loss: 0.6601 - accuracy: 0.6000 - val_loss: 0.6406 - val_accuracy: 0.6000
Epoch 6/10
5/5 [==============================] - 0s 6ms/step - loss: 0.6344 - accuracy: 0.6000 - val_loss: 0.6231 - val_accuracy: 0.6000
Epoch 7/10
5/5 [==============================] - 0s 6ms/step - loss: 0.6196 - accuracy: 0.6000 - val_loss: 0.6053 - val_accuracy: 0.6000
Epoch 8/10
5/5 [=================

In [7]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.5591359734535217
Train accuracy: 0.800000011920929


In [8]:
model.predict(x_train)

array([[0.49857116, 0.5014288 ],
       [0.6118893 , 0.3881107 ],
       [0.45008156, 0.5499185 ],
       [0.35390595, 0.646094  ],
       [0.31548625, 0.6845137 ]], dtype=float32)

In [9]:
model.predict_classes(x_train)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([1, 0, 1, 1, 1])